In [1]:
import pandas as pd

from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim 
import os

In [2]:
countries_men = pd.read_csv("countries_men.csv")
countries_pass_men = pd.read_csv("countries_pass_men.csv")
countries_shot_men = pd.read_csv("countries_shoot_men.csv")
countries_def_men = pd.read_csv("countries_def_men.csv")

countries_women = pd.read_csv("countries_women.csv")
countries_pass_women = pd.read_csv("countries_pass_women.csv")
countries_shot_women = pd.read_csv("countries_shoot_women.csv")
countries_def_women = pd.read_csv("countries_def_women.csv")

In [12]:
countries_pass_men.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 25 columns):
Squad      32 non-null object
# Pl       32 non-null int64
90s        32 non-null float64
Cmp        32 non-null int64
Att        32 non-null int64
Cmp%       32 non-null float64
TotDist    32 non-null int64
PrgDist    32 non-null int64
Cmp.1      32 non-null int64
Att.1      32 non-null int64
Cmp%.1     32 non-null float64
Cmp.2      32 non-null int64
Att.2      32 non-null int64
Cmp%.2     32 non-null float64
Cmp.3      32 non-null int64
Att.3      32 non-null int64
Cmp%.3     32 non-null float64
Ast        32 non-null int64
xA         32 non-null float64
A-xA       32 non-null float64
KP         32 non-null int64
1/3        32 non-null int64
PPA        32 non-null int64
CrsPA      32 non-null int64
Prog       32 non-null int64
dtypes: float64(7), int64(17), object(1)
memory usage: 6.4+ KB


In [3]:
countries_men_filt = countries_men[["Squad","# Pl","Age","Ast","xG","MP"]]
countries_shot_men_filt = countries_shot_men[["Squad","Gls","Sh","SoT","Dist"]]
countries_pass_men_filt = countries_pass_men[["Squad","Cmp","Att","TotDist"]]
countries_def_men_filt = countries_def_men[["Squad","Tkl","Press","Blocks"]]

countries_women_filt = countries_women[["Squad","# Pl","Age","Ast","xG","MP"]]
countries_shot_women_filt = countries_shot_women[["Squad","Gls","Sh","SoT","Dist"]]
countries_pass_women_filt = countries_pass_women[["Squad","Cmp","Att","TotDist"]]
countries_def_women_filt = countries_def_women[["Squad","Tkl","Press","Blocks"]]

full_countries_men = pd.merge(countries_men_filt, countries_shot_men_filt, on=["Squad","Squad"], how ="outer" )
full_countries_men = pd.merge(full_countries_men, countries_pass_men_filt, on=["Squad","Squad"], how ="outer" )
full_countries_men = pd.merge(full_countries_men, countries_def_men_filt, on=["Squad","Squad"], how ="outer" )

full_countries_women = pd.merge(countries_women_filt, countries_shot_women_filt, on=["Squad","Squad"], how ="outer" )
full_countries_women = pd.merge(full_countries_women, countries_pass_women_filt, on=["Squad","Squad"], how ="outer" )
full_countries_women = pd.merge(full_countries_women, countries_def_women_filt, on=["Squad","Squad"], how ="outer" )

In [4]:
#full_countries_men["Squad"] = full_countries_men.Squad.str.split(" ")
#full_countries_men["Squad"] = full_countries_men["Squad"].str[1]
full_countries_men["Sex"] = 'M'

#full_countries_women["Squad"] = full_countries_women.Squad.str.split(" ")
#full_countries_women["Squad"] = full_countries_women["Squad"].str[1]
full_countries_women["Sex"] = 'F'

result = full_countries_men.append(full_countries_women)

In [5]:
result.reset_index(drop=True, inplace=True)
result.head()

,Squad,# Pl,Age,Ast,xG,MP,Gls,Sh,SoT,Dist,Cmp,Att,TotDist,Tkl,Press,Blocks,Sex
0,Argentina,20,29.6,6,4.9,4,6,57,16,18.0,2034,2461,37880,60,634,44,M
1,Australia,15,27.7,0,3.5,3,2,32,7,17.0,1273,1552,25661,48,525,52,M
2,Belgium,21,27.7,12,11.8,7,15,107,35,16.6,3202,3779,61883,89,1432,79,M
3,Brazil,18,28.2,7,11.8,5,8,103,39,16.5,2476,2900,44288,87,868,90,M
4,Colombia,20,26.8,5,2.7,4,6,41,12,19.2,1528,1992,30540,60,754,57,M


In [6]:
#añadimos lat y long

locator = Nominatim(user_agent="myGeocoder")
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)


# 2- - create location column
result['location'] = result['Squad'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
result['point'] = result['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
result[['latitude', 'longitude', 'altitude']] = pd.DataFrame(result['point'].tolist(), index=result.index)
result.head()

,Squad,# Pl,Age,Ast,xG,MP,Gls,Sh,SoT,Dist,...,TotDist,Tkl,Press,Blocks,Sex,location,point,latitude,longitude,altitude
0,Argentina,20,29.6,6,4.9,4,6,57,16,18.0,...,37880,60,634,44,M,"(Argentina, (-34.9964963, -64.9672817))","(-34.9964963, -64.9672817, 0.0)",-34.996496,-64.967282,0.0
1,Australia,15,27.7,0,3.5,3,2,32,7,17.0,...,25661,48,525,52,M,"(Australia, (-24.7761086, 134.755))","(-24.7761086, 134.755, 0.0)",-24.776109,134.755000,0.0
2,Belgium,21,27.7,12,11.8,7,15,107,35,16.6,...,61883,89,1432,79,M,"(België / Belgique / Belgien, (50.6402809, 4.6...","(50.6402809, 4.6667145, 0.0)",50.640281,4.666715,0.0
3,Brazil,18,28.2,7,11.8,5,8,103,39,16.5,...,44288,87,868,90,M,"(Brasil, (-10.3333333, -53.2))","(-10.3333333, -53.2, 0.0)",-10.333333,-53.200000,0.0
4,Colombia,20,26.8,5,2.7,4,6,41,12,19.2,...,30540,60,754,57,M,"(Colombia, (4.099917, -72.9088133))","(4.099917, -72.9088133, 0.0)",4.099917,-72.908813,0.0


In [7]:
result = result.drop(["location","point","altitude"], axis=1)
result.head()

,Squad,# Pl,Age,Ast,xG,MP,Gls,Sh,SoT,Dist,Cmp,Att,TotDist,Tkl,Press,Blocks,Sex,latitude,longitude
0,Argentina,20,29.6,6,4.9,4,6,57,16,18.0,2034,2461,37880,60,634,44,M,-34.996496,-64.967282
1,Australia,15,27.7,0,3.5,3,2,32,7,17.0,1273,1552,25661,48,525,52,M,-24.776109,134.755000
2,Belgium,21,27.7,12,11.8,7,15,107,35,16.6,3202,3779,61883,89,1432,79,M,50.640281,4.666715
3,Brazil,18,28.2,7,11.8,5,8,103,39,16.5,2476,2900,44288,87,868,90,M,-10.333333,-53.200000
4,Colombia,20,26.8,5,2.7,4,6,41,12,19.2,1528,1992,30540,60,754,57,M,4.099917,-72.908813


In [8]:
result.to_csv("full_countries_stats.csv",index = False)